<a href="https://colab.research.google.com/github/cs-pub-ro/ML/blob/master/lab2/Laborator_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Învățare Automată
# Arbori de decizie. Păduri aleatoare
### Autori:
* Tudor Berariu - 2016
* George Muraru - 2020

## 1. Scopul laboratorului
Scopul laboratorului îl reprezintă întelegerea conceptului de arbore de decizie și implementarea unor clasificatori bazați pe acest model.

## 2. Problema de rezolvat
Problema de rezolvat ı̂n acest laborator este una de ı̂nvățare supervizată: fiind dat un **set de date X** ce conține exemple descrise printr-un set de **atribute discrete A** și etichetate cu **câte o clasă dintr-o mulțime cunoscută C**, să se construiască un model pentru clasificarea exemplelor noi.

## 3. Arbore de decizie
Un arbore de decizie este un clasificator ce aproximează funcții discrete.

Într-un arbore de decizie există 2 tipuri de noduri:
* *noduri intermediare* - conține un test pentru un atribut și are câte un arc (și implicit un subarbore) pentru fiecare valoare posibiliă a atributului
* *noduri frunză* - este etichetat cu o clasă

Pentru **a clasifica un obiect nou** se pornește din rădăcina arborelui și din fiecare nod se coboară pe arcul corespunzător valorii atributului pe care o are obiectul dat. Atunci când se ajunge ı̂ntr-un nod frunză, clasa acestuia va reprezenta predicția arborelui.

## 4. Păduri de arbori aleatori
*Pădurile de arbori aleatori* (eng. Random Forest) este un model format din mai mulți arbori de decizie.

Se bazează pe 2 hiperparametri:
* Eșantionare aleatoare din setul de date de antrenament
* Subseturi aleatoare de atribute considerate la împărțirea pe mai multi subarbori

Predicția, utilizând un astfel de model, se bazează pe clasa majoritară oferită de predicțiile indepente ale tuturor arborilor.

## 5. Workspace Setup
### Câteva biblioteci de care vom avea nevoie

In [1]:
from collections import Counter
from copy import deepcopy
import csv
from math import inf, log2
from numpy import array_split
from random import choice, shuffle

### Hiperparametrii necesari rulării

In [2]:
DATASET_NAME = 'Car'  #@param ['Chess', 'Car', 'Tennis']

# Adâncimea arborilor
D = 3 #@param {type: "slider", min: 2, max: 10}

# Procentul de exemple din setul de date utilizat la construcția arborilor
P = 100 #@param {type: "slider", min: 1, max: 100}

### Funcții ajutătoare pentru descărcarea și lucrul cu setul de date

In [3]:
class Node:
    """ Representation for a node from the decision tree """
    def __init__(self, label):
        """
            for non-leafs it is the name of the attribute
            for leafs it is the class
        """
        self.label = label
        
        # Dictionary of (attribute value, nodes)
        self.children = {}
    
    def display(self, string):
        print(string + self.label)
        string += "\t"
        if self.children:
            for key, value in self.children.items():
                print(string + key)
                value.display(string + "\t")


def getArchive(dataSetName):
    """ Checks if a specific dataset is present in the local directory, if not,
    downloads it.

    Args:
        dataSetName (str): the dataset name
    """
    datasets_url = {
        "Car": "https://raw.githubusercontent.com/cs-pub-ro/ML/master/lab/lab2/datasets/car",
        "Chess": "https://raw.githubusercontent.com/cs-pub-ro/ML/master/lab2/datasets/chess",
        "Tennis": "https://raw.githubusercontent.com/cs-pub-ro/ML/master/lab2/datasets/tennis"
    }

    assert dataSetName in datasets_url

    from os import sep, path    
    dataset_url = datasets_url[dataSetName]
    dataset_file = os.sep.join(os.path.normpath(dataset_url).split(os.sep)[-2:])
    print(dataset_file)

    if not path.isfile(dataset_file):
        import urllib
        print("Downloading...")
        urllib.request.urlretrieve(dataset_url, filename=dataset_file)
        assert(path.isfile(dataset_file))
        print("Got the archive")
    else:
        print(f"{dataset_file} already in the local directory")


def getDataSet(dataSetName):
    """ Reads a dataset

    Args:
        dataSetName (str): Name for the dataset

    Returns:
        A tuple containing (classes, attributes, examples):
        classes (set): the classes that are found in the dataset
        attributes (list of strings): the attributes for the dataset
        examples (list of dictionaries): one example contains an entry as
            (attribute name, attribute value)
    """

    dataset_file = f'datasets/{dataSetName.lower()}'

    f_in = open(dataset_file, 'r')
    csv_reader = csv.reader(f_in, delimiter=",")

    # Read the header row
    row = next(csv_reader)

    # The last element represents the class
    attributeNames = row[:-1]
    
    examples = []
    classes = set()

    for row in csv_reader:
        *attributes, label = row
        classes.add(label)
        example = dict(zip(attributeNames, attributes))
        example["CLASS"] = label
        examples.append(example)
    
    f_in.close()
    return classes, attributeNames, examples

### Descărcare și încarcare set de date

In [4]:
getArchive(DATASET_NAME)
classes, attributes, examples = getDataSet(DATASET_NAME)

print(examples[0])
print(attributes)

datasets/car
datasets/car already in the local directory
{'buying': 'vhigh', 'maint': 'vhigh', 'doors': '2', 'persons': '2', 'lug_boot': 'small', 'CLASS': 'unacc'}
['buying', 'maint', 'doors', 'persons', 'lug_boot']


## 6. Cerințe

1. [3 pct] Implementați o funcție recursivă *randomTree* care construiește arbori de decizie de adâncime **d** pe baza unui **set de date X** și a unei **mulțimi de atribute A** astfel:
 * Dacă *d = 0*, atunci se construiește un nod frunză cu clasa majoritară din X.
 * Dacă *d > 0*, atunci se alege aleator un atribut $a_i$ din A și se construiește câte un subarbore pentru fiecare valoare $v_j$ a atributului $a_i$ apelând *randomTree* pentru *d − 1*:
$$
X_{i/j} = \{x \in X|a_{i}(x) = v_k\}\\
A_{new} = A \setminus \{a_i\}
$$

In [5]:
def decisionTree(d, X, A, alg, chooser):
   if not d or not A:
      return Node(Counter([x['CLASS'] for x in X]).most_common()[0][0])
   if alg == 'id3' \
      and all(map(lambda x: x['CLASS'] == X[0]['CLASS'], X)):
      return Node(X[0]['CLASS'])

   attrib = chooser(X, A)
   Vs = set([x[attrib] for x in X])
   A_new = deepcopy(A)
   A_new.remove(attrib)

   node = Node(attrib)
   node.children = {
      v: decisionTree(
         d - 1,
         [x for x in X if x[attrib] == v],
         A_new,
         alg,
         chooser
      )
         for v in Vs
   }

   return node


def randomTree(d, X, A):
   # Cerință 1
   return decisionTree(d, X, A, 'random', lambda _, A: choice(A))


root = randomTree(2, examples, attributes)
root.display('-> ')

-> maint
-> 	high
-> 		buying
-> 			high
-> 				unacc
-> 			med
-> 				unacc
-> 			low
-> 				unacc
-> 			vhigh
-> 				unacc
-> 	med
-> 		doors
-> 			5more
-> 				unacc
-> 			2
-> 				unacc
-> 			3
-> 				unacc
-> 			4
-> 				unacc
-> 	low
-> 		lug_boot
-> 			med
-> 				unacc
-> 			big
-> 				unacc
-> 			small
-> 				unacc
-> 	vhigh
-> 		doors
-> 			5more
-> 				unacc
-> 			2
-> 				unacc
-> 			3
-> 				unacc
-> 			4
-> 				unacc


2. [3 pct] Implementați o funcție recursivă *id3* care construiește arbori de decizie pe baza unui **set de date X** și a unei **mulțimi de atribute A**.
    
  Trebuie avute în vedere următoarele aspecte:
  * dacă toate exemplele din X aparțin unei singure clase C, atunci se construiește un nod frunză etichetat cu acea clasă C
  * dacă nu mai exista atribute, atunci construiește nodul frunză etichetat cu cea mai frecventă clasă din X
    
  În caz contrar:
  * se alege atributul $a^*$ care aduce cel mai mai mare câștig informațional și se construiește un *nod intermediar* corespunzător acestuia.

  $$
    entropy(X) = -\sum_{c \in C}\frac{|X_c|}{|X|}log_2\frac{|X_c|}{|X|}
  $$
  $$
    gain(X, a) = entropy(X) - \sum_{v_{j} \in vals(a)} \frac{|X_{i/j}|}{|X|}entropy(X_{i/j})
  $$
  $$
    a^* = \underset{a \in A}{\operatorname{arg max}}\ gain(X, a)
  $$

  * pentru fiecare valoare posibilă $v_j$ a lui $a^*$ se construiește un subarbore apelând recursiv funcția *id3* pentru:

$$
  X_j = \{x \in X|a^*(x) = v_j\}\\
  A_{new} = A\setminus\{a^*\}
$$

În cazul prezentat mai sus, entropia este utilizată pentru a măsura randomness-ul din date. Intuitiv, cu cât un eveniment are probabilitate mai mare să se întâmple atunci acesta va avea o entropia din ce în ce mai mică. Prin modul în care se construiește arborele *ID3* se încearcă reducerea entropiei alegând la fiecare pas atributele care ne ofera cea mai multă informație. Cât considerați că este entropia într-un *nod frunză*?

In [6]:
def mostFrequentClass(X):
    # TODO Cerință 2
    return Counter([x['CLASS'] for x in X]).most_common()[0][0]


def entropy(X):
    # TODO Cerință 2
    classes = {}
    for x in X:
        if x['CLASS'] in classes:
            classes[x['CLASS']] += 1
        else:
            classes[x['CLASS']] = 1

    return -sum(num / len(X) * log2(num / len(X))
        for _, num in classes.items())


def gain(X, a):
    # TODO Cerință 2
    Vs = set([x[a] for x in X])
    l = len(X)
    gain = entropy(X)

    for v in Vs:
        X_new = [x for x in X if x[a] == v]
        gain -= len(X_new) * entropy(X_new) / l

    return gain  


def getMaxGainAttrib(X, A):
    max_gain = -inf

    for attr in A:
        crt_gain = gain(X, attr)
        if crt_gain > max_gain:
            max_gain = crt_gain
            attrib = attr

    return attrib


def id3(X, A):
    # TODO Cerință 2
    return decisionTree(-1, X, A, 'id3', getMaxGainAttrib)


def evaluate(tree, example):
    # TODO Cerință 2
    # Functia intoarce clasa prezisa de arborele `tree` pentru exemplul `example`
    if not tree.children:
        return tree.label
    if example[tree.label] not in tree.children:
        return evaluate(list(tree.children.values())[0], example)
    return evaluate(tree.children[example[tree.label]], example)

3. [4 pct] Implementați clasificator de tip pădure de arbori aleatori construind *n* arbori de adâncime maximă *d* fiecare dintre aceștia pornind de la o submulțime aleatoare a lui X.

    Folosiți funcția *randomTree* de la cerința 1.
  * Porniți de la *n = 100*, *d = 3* și submulțimi formate din 50% din elementele lui X alese la întamplare și experimentați cu acești hiperparametri.
  * Pentru predicția clasei pentru obiecte noi alegeți clasa majoritară
  * Comparați rezultatele obținute folosind un singur arbore construit cu ID3 și o pădure de arbori aleatori. Discuție după *zgomot*, *overfitting*.

In [7]:
def randomForest(X, A, n, d):
    # TODO Cerință 3
    shuffle(X)
    return [randomTree(d, list(chunk), A) for chunk in array_split(X, n)]


def randomID3Forest(X, A, n):
    shuffle(X)
    return [id3(list(chunk), A) for chunk in array_split(X, n)]


def evaluateForest(forest, x):
    return Counter(list(map(lambda t: evaluate(t, x), forest))).most_common()[0][0]


def precision(tree, X, c, type):
    prec = 0
    predicted_ct = 0
    evaluator = evaluate if type == 'tree' else evaluateForest
    
    for ex in X:
        pred_c = evaluator(tree, ex)
        if pred_c == c:
            predicted_ct += 1
            if ex['CLASS'] ==c:
                prec += 1
    
    if predicted_ct != 0:
        return prec / predicted_ct
    return 0


def recall(tree, X, c, type):
    X_c = list(filter(lambda ex: ex['CLASS'] == c, X))
    recall = 0
    evaluator = evaluate if type == 'tree' else evaluateForest
    
    for ex in X_c:
        pred_c = evaluator(tree, ex)
        if pred_c == c:
            recall += 1
            
    recall /= len(X_c)
    return recall


def accuracy(tree, X, type):
    count = 0
    evaluator = evaluate if type == 'tree' else evaluateForest

    for x in X:
        if evaluator(tree, x) == x['CLASS']:
            count += 1
    
    return 1.0 * count / len(X)


def test_algs(root, X, type, tabs):
    for clss in set([x['CLASS'] for x in X]):
        print(tabs * '\t' + f'prec for class {clss} = {precision(root, X, clss, type)}')
        print(tabs * '\t' + f'recall for class {clss} = {recall(root, X, clss, type)}')
        print()
    print(tabs * '\t' + f'acc = {accuracy(root, X, type)}')

In [8]:
DATASETS = ['Car', 'Chess', 'Tennis']

for ds in DATASETS:
    classes, attributes, examples = getDataSet(ds)
    shuffle(examples)

    n_X = len(examples)

    train_X = examples[:int(n_X * .8)]
    test_X = examples[int(n_X * .8):]
    print(f'Testing dataset {ds}')

    print(f'\tTesting randomTree:')
    for d in range(2, 6):
        print(f'\t\td = {d}:')
        tree = randomTree(d, train_X, attributes)
        test_algs(tree, test_X, 'tree', 2)
        print()
    
    print(f'\tTesting ID3:')
    tree = id3(train_X, attributes)
    test_algs(tree, test_X, 'tree', 1)
    print()

    print('\tTesting randomForest with randomTrees:')
    for n in range(10, 100, 10):
        print(f'\t\tn = {n}')
        for d in range(2, 6):
            print(f'\t\t\td = {d}:')
            forest = randomForest(train_X, attributes, n, d)
            test_algs(forest, test_X, 'forest', 3)
            print()

Testing dataset Car
	Testing randomTree:
		d = 2:
		prec for class acc = 0
		recall for class acc = 0.0

		prec for class vgood = 0
		recall for class vgood = 0.0

		prec for class good = 0
		recall for class good = 0.0

		prec for class unacc = 0.708092485549133
		recall for class unacc = 1.0

		acc = 0.708092485549133

		d = 3:
		prec for class acc = 0.3333333333333333
		recall for class acc = 0.23943661971830985

		prec for class vgood = 0
		recall for class vgood = 0.0

		prec for class good = 0
		recall for class good = 0.0

		prec for class unacc = 0.7491525423728813
		recall for class unacc = 0.9020408163265307

		acc = 0.6878612716763006

		d = 4:
		prec for class acc = 0.3783783783783784
		recall for class acc = 0.39436619718309857

		prec for class vgood = 0.08333333333333333
		recall for class vgood = 0.07142857142857142

		prec for class good = 0.0
		recall for class good = 0.0

		prec for class unacc = 0.765625
		recall for class unacc = 0.8

		acc = 0.6502890173410405

		

## Concluzii
Se observa ca, in cazul modelului `randomForest`, pentru majoritatea valorilor lui `n` exista o adancime `d` a arborilor din padure, astfel incat acuratetea modelului sa fie maxima. De exemplu, pentru datasetul **Chess** si `n = 20`, adancimea optima este 4, pentru care acuratetea modelului este de 0.795. De la `d = 5` apare fenomenul de *overfitting*, iar, pentru  `d <= 3`, acuratetea scade din nou, fiind vorba despre *underfitting*.

De asemenea, unele dataset-uri, cum ar fi **Car**, se satureaza rapid, toate modelele avand acurateti similare (~70%), chiar si `randomForest`, indiferent de `n` si `d`. Pe deasupra, se poate spune ca, pe masura ce crestem `d`, `randomTree` tinde sa se comporte tot mai mult ca `id3`. Cu cat adancimea este mai mare, `randomTree` are posibilitatea de a decide intr-un mod mai complex, asemanator lui `id3`. Cu aceasta complexitate, creste si riscul de *overfitting* vizibil pentru datasetul **Car**, unde cea mai buna acuratete se obtine pentru `d = 2`.

Un alt fenomen observat la datasetul **Tennis** este cauzat de impartirea exemplelor in seturi de antrenare (80%) si testare (20%). Este posibli ca unele clase sa nu fie deloc prezente in unul dintre seturi, drept care valorile preciziei, regasirii si acuratetii sunt 0 pentru anumite valori ale lui `n` si `d` in cazul `randomForest`.

## 7. Set de date
În cadrul acestui laborator se vor folosi seturile de date [Car Evaluation](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation), [Chess](https://archive.ics.uci.edu/ml/datasets/Chess+%28King-Rook+vs.+King-Pawn%29) și [Tennis](https://www.kaggle.com/fredericobreno/play-tennis).

Aceste seturi de date sunt "ușor" modificate astfel încât pe prima linie să se afle atributele și labelul/clasa din care face parte fiecare exemplu.

Atributele datasetului *Chess* nu sunt intuitive, iar dacă doriți să aflați mai multe informații despre acestea, puteți accesa link-ul de [aici](https://pdfs.semanticscholar.org/db58/88d3f373aff2c6bd7b2f956b81c6896874a9.pdf?_ga=2.193733611.798337455.1582711694-486327444.1582711694).

## 8. Extra
### 8.1 ID3 exemplu
Un exemplu mai detaliat pentru construcția arborelui de decizie ID3 se poate găsi [aici](https://github.com/cs-pub-ro/ML/blob/master/lab/lab2/id3_example.pdf).

### 8.2 CART
Un alt algoritm utilizat poartă denumirea de CART (eng. Classification and Regression Tree). Dacă **ID3** utilizeaza **câștigul informațional (eng. information gain)**, **CART** utilizeaza o altă metrică numită **index-ul Gini (eng. Gini index sau Gini impurity)**.

Pentru implementare, se urmăresc exact aceeași [pași ca la ID3](#scrollTo=rjYqUPSbe1gG), singura diferentă fiind modul în care se calculează atributul utilizat într-un *nod intermediar*.
$$
Gini(X, a) = 1 - \sum_{c \in C}{p(c | attr(X) = a) ^2}
\\
a^* = \underset{a \in A}{\operatorname{arg min}}\ Gini(X, a)
$$